In [27]:
import numpy as np
import tensorflow as tf
import time
from tensorflow.python.client import device_lib

import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

In [28]:
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 13543796489064384346
]


In [43]:
ITER = 500


# Numpy Performance

In [48]:
X = np.random.randn(1000,1000)
start = time.time()
for i in range(ITER):
    out = np.matmul(X,X)
end = time.time()
print("Time Taken: ",end - start)

Time Taken:  27.638689041137695


# Tensorflow Performance

In [45]:
X = tf.placeholder(shape = (1000,1000),dtype=tf.float32)
start = time.time()
for i in range(ITER):
    out = tf.matmul(X,X)
end = time.time()
print("Time Taken: ",end - start)

Time Taken:  0.33898305892944336


In [46]:
with tf.device('/device:CPU:0'):
    start = time.time()
    for i in range(ITER):
        out = tf.matmul(X,X)
    end = time.time()
print("Time Taken : ",end-start)

Time Taken :  0.32958102226257324


In [49]:
x = tf.placeholder(shape = (1000,1000),dtype=tf.float32)
out = tf.matmul(x,x)
with tf.device('/device:CPU:0'):
    sess = tf.Session()
    start = time.time()
    for i in range(ITER):
        sess.run(out,feed_dict={
            x:X
        })
    end = time.time()
print("Time Taken : ",end-start)

Time Taken :  18.025968074798584
